In [1]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/tort-siamese
!mkdir -p data
!mkdir -p tmp
!mkdir -p checkpoints
!mkdir -p pretrained
!mkdir -p results
!pwd

Mounted at /content/gdrive
/content/gdrive/MyDrive/tort-siamese
/content/gdrive/MyDrive/tort-siamese


In [3]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [4]:
print(tf.__version__)

2.7.0


In [ ]:
def multi_input_proportional_generator(datasets,
                                       label,
                                       p=[0.1, 0.9],
                                       batch_size=128):
    # p indicate number of class and sampling prob
    while (True):
        batch_data = [[], []]
        batch_label = []
        sample_id = np.random.choice(len(p), batch_size, p=p)
        query_idx = [
            np.where(label == class_id)[0] for class_id in range(len(p))
        ]
        for class_id in sample_id:
            query_id = np.random.choice(query_idx[class_id], 1)[0]
            batch_data[0].append(datasets[0][query_id])
            batch_data[1].append(datasets[1][query_id])
            batch_label.append(label[query_id])
        batch_data[0] = np.array(batch_data[0])
        batch_data[1] = np.array(batch_data[1])
        yield batch_data, np.array(batch_label)

In [ ]:
def process_input(num_words, X_train, X_test, max_sequnce_len = 1000):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=num_words, oov_token='<UNK>')
    tokenizer.fit_on_texts(X_train)

    #convert text data to numerical indexes
    train_seqs = tokenizer.texts_to_sequences(X_train)
    test_seqs = tokenizer.texts_to_sequences(X_test)

    # max_sequnce_len = max([len(x) for x in train_seqs])

    train_seqs = tf.keras.preprocessing.sequence.pad_sequences(train_seqs, maxlen=max_sequnce_len, padding="post")
    test_seqs=tf.keras.preprocessing.sequence.pad_sequences(test_seqs, maxlen=max_sequnce_len, padding="post")

    return train_seqs, test_seqs, max_sequnce_len

In [ ]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    sqaure_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * sqaure_pred + (1 - y_true) * margin_square)

In [ ]:
def base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    num_words = 1000
    embedding_size = 300
    input = tf.keras.layers.Input(shape=input_shape)
    # x = tf.keras.layers.Flatten()(input)
    x = tf.keras.layers.Embedding(num_words, embedding_size, trainable=True)(input)
    # x = tf.keras.layers.Dense(128, activation='relu')(x)
    # x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = tf.keras.layers.Lambda(lambda  x: K.l2_normalize(x,axis=1))(x)
    x = tf.keras.layers.Lambda(lambda  x: K.l2_normalize(x,axis=1))(x)
    return tf.keras.models.Model(input, x)

In [5]:
df = pd.read_pickle('./data/processed_torts20210321.pkl')
df.head()

,legal_encoded,case_id,plaintiff_token,defendant_token,legal_name,legal_section,legal_content_token
0,687,2539###439,"[โจทก์, ฟ้อง, ว่า, , เดิม, โจทก์, ถูก, ฟ้อง, ...","[จำเลย, ที่, , 1, , ให้การ, ว่า, , โจทก์, ไ...",ประมวลกฎหมายแพ่งและพาณิชย์,420,"[ผู้, ใด, จงใจ, หรือ, ประมาท, เลินเล่อทำ, ต่อ,..."
1,251,2539###439,"[โจทก์, ฟ้อง, ว่า, , เดิม, โจทก์, ถูก, ฟ้อง, ...","[จำเลย, ที่, , 1, , ให้การ, ว่า, , โจทก์, ไ...",ประมวลกฎหมายวิธีพิจารณาความแพ่ง,55,"[เมื่อ, มี, ข้อ, โต้แย้ง, เกิด, ขึ้น, เกี่ยว, ..."
2,257,2539###439,"[โจทก์, ฟ้อง, ว่า, , เดิม, โจทก์, ถูก, ฟ้อง, ...","[จำเลย, ที่, , 1, , ให้การ, ว่า, , โจทก์, ไ...",ประมวลกฎหมายวิธีพิจารณาความแพ่ง,60,"[คู่, ความ, ฝ่าย, ใด, ฝ่าย, หนึ่ง, หรือ, ผู้, ..."
3,259,2539###439,"[โจทก์, ฟ้อง, ว่า, , เดิม, โจทก์, ถูก, ฟ้อง, ...","[จำเลย, ที่, , 1, , ให้การ, ว่า, , โจทก์, ไ...",ประมวลกฎหมายวิธีพิจารณาความแพ่ง,62,"[ทนายความ, ซึ่ง, คู่, ความ, ได้, ตั้ง, แต่ง, น..."
4,135,2539###439,"[โจทก์, ฟ้อง, ว่า, , เดิม, โจทก์, ถูก, ฟ้อง, ...","[จำเลย, ที่, , 1, , ให้การ, ว่า, , โจทก์, ไ...",ประมวลกฎหมายวิธีพิจารณาความแพ่ง,142,"[คำ, พิพากษา, หรือ, คำ, สั่ง, ของ, ศาล, ที่, ช..."


In [6]:
input_legth = df.defendant_token.map(len)

In [11]:
input_legth = df.legal_content_token.map(len)

In [10]:
input_legth = df.plaintiff_token.map(len)

In [12]:
np.max(input_legth), np.min(input_legth), np.mean(input_legth)

(1232, 3, 98.70926602457655)

In [13]:
df.legal_content_token

0        [ผู้, ใด, จงใจ, หรือ, ประมาท, เลินเล่อทำ, ต่อ,...
1        [เมื่อ, มี, ข้อ, โต้แย้ง, เกิด, ขึ้น, เกี่ยว, ...
2        [คู่, ความ, ฝ่าย, ใด, ฝ่าย, หนึ่ง, หรือ, ผู้, ...
3        [ทนายความ, ซึ่ง, คู่, ความ, ได้, ตั้ง, แต่ง, น...
4        [คำ, พิพากษา, หรือ, คำ, สั่ง, ของ, ศาล, ที่, ช...
                               ...                        
15050    [ผู้, ใด, จงใจ, หรือ, ประมาท, เลินเล่อทำ, ต่อ,...
15051    [ผู้, ใด, จงใจ, หรือ, ประมาท, เลินเล่อทำ, ต่อ,...
15052    [เมื่อ, มี, ข้อ, โต้แย้ง, เกิด, ขึ้น, เกี่ยว, ...
15053    [ผู้, ใด, จงใจ, หรือ, ประมาท, เลินเล่อทำ, ต่อ,...
15054    [บุคคล, ตั้งแต่, สอง, คน, ขึ้น, ไป,  , อาจ, เป...
Name: legal_content_token, Length: 15055, dtype: object

In [ ]:
input_legth = df.defendant_token.map(len)

In [16]:
# df.plaintiff_token.map(len)
df[df.legal_content_token.map(len) == 3]

,legal_encoded,case_id,plaintiff_token,defendant_token,legal_name,legal_section,legal_content_token
4444,380,2544###6340,"[คดี, ทั้ง, หก, สำนวน, นี้, ศาลชั้นต้น, มี, คำ...","[โจทก์, ทั้ง, หก, สำนวน, ฟ้อง, ขอให้, จำเลย, ท...",ประมวลกฎหมายแพ่งและพาณิชย์,1246,"[(, ยกเลิก, )]"
4456,380,2544###6340,"[คดี, ทั้ง, หก, สำนวน, นี้, ศาลชั้นต้น, มี, คำ...",[],ประมวลกฎหมายแพ่งและพาณิชย์,1246,"[(, ยกเลิก, )]"
5415,176,2539###843,"[โจทก์, , ฟ้อง, , ว่า, , โจทก์, , เป็น, ,...","[ศาลชั้นต้น, , พิเคราะห์, , คำฟ้อง, , แล้ว,...",ประมวลกฎหมายวิธีพิจารณาความแพ่ง,208,"[(, ยกเลิก, )]"
5421,176,2539###843,"[โจทก์, ฟ้อง, ว่า, , โจทก์, เป็นเจ้าของ, กรรม...","[ศาลชั้นต้น, พิเคราะห์, คำฟ้อง, แล้ว, , มี, ค...",ประมวลกฎหมายวิธีพิจารณาความแพ่ง,208,"[(, ยกเลิก, )]"


In [ ]:
''.join(df.iloc[15054].plaintiff_token), df.iloc[15054].case_id

('โจทก์ฟ้องขอให้เพิกถอนโฉนดที่ดินเลขที่ 1309 ตำบลโซ่ อำเภอโซ่พิสัย จังหวัดหนองคาย (บึงกาฬ) เนื้อที่ 10 ไร่ 1 งาน 14 ตารางวา',
 '2560###1191')

In [ ]:
np.mean(df[df['legal_section'] == '55'].plaintiff_token.map(len))

200.02351097178683

In [ ]:
df[df['legal_section'] == '55'].plaintiff_token.map(len)

1        488
7        403
76       451
102       99
105      125
        ... 
14988     45
14994    228
15012    149
15026     31
15052     43
Name: plaintiff_token, Length: 638, dtype: int64

In [ ]:
df.iloc[15054].case_id, ''.join(df.iloc[105].plaintiff_token)

('2560###1191',
 'โจทก์ฟ้องว่า โจทก์ใช้สิทธิครอบครองทำประโยชน์ที่ดินมือเปล่า โดยปลูกพืชไร่ ทำสวนผักตามฤดูกาล และปลูกต้นมะพร้าวมะม่วง มะขาม มานานกว่า 5 ปีแล้ว จำเลยทั้งสี่ได้ร่วมกันเข้าไปไถปรับหน้าที่ดินโจทก์ที่ครอบครองอยู่ทางด้านทิศเหนือและทิศใต้โดยไม่ได้รับความยินยอมจากโจทก์เป็นเนื้อที่ประมาณ 1 ไร่ 2 งานอันเป็นการรบกวนสิทธิของโจทก์ ทำให้โจทก์เสียหาย ขอให้บังคับห้ามมิให้จำเลยทั้งสี่เข้ารบกวนสิทธิครอบครองในที่ดินที่โจทก์ใช้สิทธิครอบครองอยู่ ให้จำเลยทั้งสี่ร่วมกันหรือแทนกันใช้ค่าเสียหายแก่โจทก์ จนกว่าจำเลยทั้งสี่จะเลิกการรบกวนสิทธิของโจทก์')

In [ ]:
df.iloc[14994].case_id, ''.join(df.iloc[14994].plaintiff_token)

('2558###9797',
 'โจทก์ทั้งสามฟ้องและแก้ไขคำฟ้องโดยได้รับอนุญาตให้ยกเว้นค่าธรรมเนียมในศาลชั้นต้นห้ามจำเลยทั้งสี่และบริวารเข้ามายุ่งเกี่ยวรบกวนสิทธิครอบครองที่ดินพิพาทของโจทก์ทั้งสาม ให้จำเลยทั้งสี่และบริวารรื้อถอนโรงเรือนที่สร้างขึ้นในที่ดินของโจทก์ที่ 2 ให้จำเลยทั้งสี่ร่วมกันหรือแทนกันชดใช้ค่าต้นปาล์มน้ำมันและรั้วแก่โจทก์ที่ 1 เป็นเงิน 1,286,000 บาท ชดใช้ค่าบ้านและทรัพย์สินที่ถูกทำลายแก่โจทก์ที่ 3 เป็นเงิน 3,353,500 บาท ชดใช้ค่าต้นปาล์มน้ำมัน หมาก มะพร้าว และกล้วยแก่โจทก์ที่ 1 และที่ 2 เป็นเงิน 46,500 บาท พร้อมดอกเบี้ยร้อยละ 7.5  ต่อปี จากต้นเงินแต่ละจำนวนนับแต่วันฟ้องจนกว่าจะชำระเสร็จแก่โจทก์แต่ละคน กับให้จำเลยทั้งสี่ร่วมกันหรือแทนกันชดใช้ค่าเสียหายแก่โจทก์ที่ 1 เป็นรายเดือน เดือนละ 20,000 บาท และแก่โจทก์ที่ 2 เป็นรายเดือนเดือนละ 10,000 บาท นับแต่วันที่ 1 กุมภาพันธ์ 2552 จนกว่าจำเลยทั้งสี่และบริวารจะรื้อถอนโรงเรือนออกจากที่ดินพิพาทและเลิกเข้ายุ่งเกี่ยวรบกวนครอบครองที่ดินพิพาทของโจทก์ทั้งสาม')

In [ ]:
x55 = df[df['legal_section'] == '55']

In [ ]:
x55[x55.plaintiff_token.map(len) > 200]

,legal_encoded,case_id,plaintiff_token,defendant_token,legal_name,legal_section,legal_content_token
1,251,2539###439,"[โจทก์, ฟ้อง, ว่า, , เดิม, โจทก์, ถูก, ฟ้อง, ...","[จำเลย, ที่, , 1, , ให้การ, ว่า, , โจทก์, ไ...",ประมวลกฎหมายวิธีพิจารณาความแพ่ง,55,"[เมื่อ, มี, ข้อ, โต้แย้ง, เกิด, ขึ้น, เกี่ยว, ..."
7,251,2539###439,"[โจทก์, , ฟ้อง, , ว่า, , เดิม, , โจทก์, ,...","[จำเลย, , ที่, , 1, , ที่, , 2, , ให้การ,...",ประมวลกฎหมายวิธีพิจารณาความแพ่ง,55,"[เมื่อ, มี, ข้อ, โต้แย้ง, เกิด, ขึ้น, เกี่ยว, ..."
76,251,2539###5684,"[โจทก์, ทั้งสอง, สำนวน, ฟ้อง, ว่า, , โจทก์, เ...","[จำเลย, ทั้งสอง, สำนวน, ให้การ, ว่า, , จำเลย,...",ประมวลกฎหมายวิธีพิจารณาความแพ่ง,55,"[เมื่อ, มี, ข้อ, โต้แย้ง, เกิด, ขึ้น, เกี่ยว, ..."
137,251,2533###472,"[โจทก์, ฟ้อง, ว่า, , โจทก์, เป็น, ผู้, ถือ, ก...","[ศาลฎีกา, วินิจฉัย, ว่า, , "", คดี, มีปัญหา, ว...",ประมวลกฎหมายวิธีพิจารณาความแพ่ง,55,"[เมื่อ, มี, ข้อ, โต้แย้ง, เกิด, ขึ้น, เกี่ยว, ..."
161,251,2538###1892,"[โจทก์, , ฟ้อง, , ว่า, , โจทก์, , รับประกั...","[จำเลย, , ให้การ, , ว่า, , เหตุ, , ที่, ,...",ประมวลกฎหมายวิธีพิจารณาความแพ่ง,55,"[เมื่อ, มี, ข้อ, โต้แย้ง, เกิด, ขึ้น, เกี่ยว, ..."
...,...,...,...,...,...,...,...
14486,251,2538###693,"[โจทก์, ฟ้อง, ว่า, , โจทก์, เป็น, นิติบุคคล, ...","[จำเลย, ให้การ, ว่า, , การ, ขอ, ใช้, ชื่อ, นิ...",ประมวลกฎหมายวิธีพิจารณาความแพ่ง,55,"[เมื่อ, มี, ข้อ, โต้แย้ง, เกิด, ขึ้น, เกี่ยว, ..."
14491,251,2538###8177,"[โจทก์, ฟ้อง, ว่า, , โจทก์, ที่, , 1, , เป็...","[จำเลย, ทั้ง, สี่, ให้การ, ว่า, , ที่ดิน, โฉน...",ประมวลกฎหมายวิธีพิจารณาความแพ่ง,55,"[เมื่อ, มี, ข้อ, โต้แย้ง, เกิด, ขึ้น, เกี่ยว, ..."
14494,251,2538###7985,"[โจทก์, ฟ้อง, ว่า, , เมื่อ, วันที่, , 1, , ...","[จำเลย, ให้การ, ว่า, , การ, ที่, นางสาว, วีณา...",ประมวลกฎหมายวิธีพิจารณาความแพ่ง,55,"[เมื่อ, มี, ข้อ, โต้แย้ง, เกิด, ขึ้น, เกี่ยว, ..."
14599,251,2540###3047,"[โจทก์, ฟ้อง, และ, แก้ไข, คำฟ้อง, ว่า, , โจทก...","[จำเลย, ให้การ, ว่า, , โจทก์, ไม่, มี, สิทธิค...",ประมวลกฎหมายวิธีพิจารณาความแพ่ง,55,"[เมื่อ, มี, ข้อ, โต้แย้ง, เกิด, ขึ้น, เกี่ยว, ..."


In [ ]:
''.join(df.iloc[15052].plaintiff_token), df.iloc[15052].case_id

('โจทก์ฟ้องและแก้ไขคำฟ้องขอให้บังคับจำเลยชดใช้เงินจำนวน 9,295,745.56 บาท พร้อมดอกเบี้ยในอัตราร้อยละ 7.5 ต่อปี ของต้นเงินจำนวน 8,742,455.89 บาท นับแต่วันฟ้องจนกว่าจะชำระเสร็จแก่โจทก์',
 '2560###294')

In [ ]:
.plaintiff_token.map(''.join)

In [ ]:
df[df.plaintiff_token.map(len) == 3].plaintiff_token.map()

In [ ]:
df.defendant_token.map(len)

0        449
1        449
2        449
3        449
4        449
        ... 
15050      8
15051      5
15052      5
15053     19
15054     19
Name: defendant_token, Length: 15055, dtype: int64

In [ ]:
df.defendant_token.map(len)

In [ ]:
df[df.defendant_token.map(len) > 70].defendant_token

0        [จำเลย, ที่,  , 1,  , ให้การ, ว่า,  , โจทก์, ไ...
1        [จำเลย, ที่,  , 1,  , ให้การ, ว่า,  , โจทก์, ไ...
2        [จำเลย, ที่,  , 1,  , ให้การ, ว่า,  , โจทก์, ไ...
3        [จำเลย, ที่,  , 1,  , ให้การ, ว่า,  , โจทก์, ไ...
4        [จำเลย, ที่,  , 1,  , ให้การ, ว่า,  , โจทก์, ไ...
                               ...                        
15015    [\t, ต่อมา, โจทก์, ยื่นคำร้อง, ว่า,  , โจทก์, ...
15041    [   , จำเลย, ที่,  , 1,  , ให้การ, ขอให้, ยกฟ้...
15042    [   , จำเลย, ที่,  , 1,  , ให้การ, ขอให้, ยกฟ้...
15043    [   , จำเลย, ที่,  , 1,  , ให้การ, ขอให้, ยกฟ้...
15044    [   , จำเลย, ที่,  , 1,  , ให้การ, ขอให้, ยกฟ้...
Name: defendant_token, Length: 4809, dtype: object

In [ ]:
''.join(df.iloc[15043].defendant_token)

'   จำเลยที่ 1 ให้การขอให้ยกฟ้อง และฟ้องแย้งขอให้เพิกถอนสิทธิบัตรกรรมวิธีการกระตุ้นให้สร้างสาร Aquilaria resin โดยการสร้างลักษณะรอยแผลบนต้นกฤษณา (Aquilaria) เลขที่ 18985 และแจ้งคำสั่งให้นายทะเบียนสิทธิบัตรเพิกถอนสิทธิบัตรดังกล่าวออกจากสารบบของสำนักงานสิทธิบัตร ให้โจทก์ใช้ค่าเสียหายเป็นเงิน 600,000,000 บาท พร้อมดอกเบี้ยอัตราร้อยละ 7.5 ต่อปี นับถัดจากวันฟ้องแย้งจนกว่าโจทก์จะชำระค่าเสียหายเสร็จ และค่าเสียหายรายเดือน เดือนละ 500,000 บาท นับถัดจากวันฟ้องแย้งจนกว่าโจทก์จะได้ปฏิบัติตามฟ้องแย้งแก่จำเลยที่ 1 ให้โจทก์ลงโฆษณาคำพิพากษาของศาลทั้งฉบับในหนังสือพิมพ์รายวัน ที่แพร่หลายอย่างน้อยจำนวน 3 ฉบับ เป็นเวลา 3 วัน ติดต่อกัน ด้วยค่าใช้จ่ายของโจทก์ และให้โจทก์ส่งสำเนาคำพิพากษาซึ่งรับรองสำเนาถูกต้องไปยังลูกค้าของจำเลยที่ 1 ทางไปรษณีย์ลงทะเบียนตอบรับด้วยค่าใช้จ่ายของโจทก์ '

In [ ]:
np.max(input_legth), np.min(input_legth), np.mean(input_legth)

(1439, 0, 77.71012952507472)

In [ ]:
label = 692
df[df['legal_encoded'] == label].head()

,legal_encoded,case_id,plaintiff_token,defendant_token,legal_name,legal_section,legal_content_token
15,692,2539###491,"[โจทก์, , ฟ้อง, , ว่า, , จำเลย, , ที่, , ...","[จำเลย, , ที่, , 1, , ขาด, นัด, , ยื่น, คำ...",ประมวลกฎหมายแพ่งและพาณิชย์,425,"[นาย, จ้าง, ต้อง, ร่วม, กัน, รับ, ผิด, กับ, ลู..."
19,692,2539###491,"[โจทก์, , ฟ้อง, , ว่า, , เมื่อ, , วันที่, ...","[จำเลย, , ที่, , 1, , ขาด, นัด, , ยื่น, คำ...",ประมวลกฎหมายแพ่งและพาณิชย์,425,"[นาย, จ้าง, ต้อง, ร่วม, กัน, รับ, ผิด, กับ, ลู..."
23,692,2539###491,"[โจทก์, ฟ้อง, ว่า, , เมื่อ, วันที่, , 16, ,...","[จำเลย, ที่, , 1, , ขาด, นัด, ยื่น, คำให้การ...",ประมวลกฎหมายแพ่งและพาณิชย์,425,"[นาย, จ้าง, ต้อง, ร่วม, กัน, รับ, ผิด, กับ, ลู..."
29,692,2539###628,"[คดี, , สอง, , สำนวน, , นี้, , ศาลชั้นต้น,...","[สำนวน, , แรก, , โจทก์, , ฟ้อง, , และ, , ...",ประมวลกฎหมายแพ่งและพาณิชย์,425,"[นาย, จ้าง, ต้อง, ร่วม, กัน, รับ, ผิด, กับ, ลู..."
40,692,2539###628,"[คดี, สอง, สำนวน, นี้, ศาลชั้นต้น, พิจารณา, พิ...","[สำนวน, แรก, , โจทก์, ฟ้อง, ว่า, , จำเลย, ที...",ประมวลกฎหมายแพ่งและพาณิชย์,425,"[นาย, จ้าง, ต้อง, ร่วม, กัน, รับ, ผิด, กับ, ลู..."


In [ ]:
positive_df = df[df['legal_encoded'] == label]
negative_df = df[~df.case_id.isin(positive_df.case_id)]

In [ ]:
x2 = df[df['legal_encoded'] == label].iloc[0].legal_content_token
positve_X1 = positive_df.plaintiff_token.values
negative_X1 = negative_df.plaintiff_token.values
Y = []
X = []
for x in positve_X1:
    X.append([x, x2])
    Y.append([1.])
for x in negative_X1:
    X.append([x, x2])
    Y.append([0.])
X = np.array(X, dtype=object)
Y = np.array(Y)

In [ ]:
X.shape, Y.shape

((13200, 2), (13200, 1))

In [ ]:
BATCH_SIZE=64
EPOCHS=10
NUM_WORDS=1000

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
train_seqs_1, test_seqs_1, max_sequnce_len_1 = process_input(NUM_WORDS, X_train[:, 0], X_test[:, 0])
train_seqs_2, test_seqs_2, max_sequnce_len_2 = process_input(NUM_WORDS, X_train[:, 1], X_test[:, 1])

train_generator = multi_input_proportional_generator([train_seqs_1, train_seqs_2], Y_train, p=[0.5, 0.5], batch_size=BATCH_SIZE)
validation_generator = multi_input_proportional_generator([test_seqs_1, test_seqs_2], Y_test, batch_size=BATCH_SIZE)

In [ ]:
input_shape = max(max_sequnce_len_1, max_sequnce_len_2)

In [ ]:
base_network = base_network(input_shape)

input_a = tf.keras.layers.Input(shape=input_shape)
input_b = tf.keras.layers.Input(shape=input_shape)

In [ ]:

def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.5
    print(y_true.shape, y_pred.shape, pred.shape)
    return np.mean(pred == y_true)


def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))

In [ ]:

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall


def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision


def microf1(y_true, y_pred):

    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))


def macrof1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true * y_pred, 'float'), axis=0)
    # tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2 * p * r / (p + r + K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

In [ ]:
# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a = base_network(input_a)
processed_b = base_network(input_b)

distance = tf.keras.layers.Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_a, processed_b])

model =  tf.keras.models.Model([input_a, input_b], distance)

# train
rms = tf.keras.optimizers.RMSprop()
#rms = Adam()
#rms = SGD()

model.compile(loss=contrastive_loss, optimizer=rms, metrics=[accuracy])


In [ ]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 1000)]       0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 1000)]       0                                            
__________________________________________________________________________________________________
model (Functional)              (None, 1000, 128)    355040      input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 1, 128)       0           model[0][0]                

In [ ]:
# num_batches = int(len(train_seqs_1)/BATCH_SIZE)
history = model.fit([train_seqs_1, train_seqs_2], Y_train,
          batch_size=128,
          epochs=10,
          validation_data=([test_seqs_1, test_seqs_2], Y_test))

Epoch 1/10
83/83 [==============================] - 16s 178ms/step - loss: 0.0562 - accuracy: 0.9503 - val_loss: 0.1329 - val_accuracy: 0.8244
Epoch 2/10
83/83 [==============================] - 14s 172ms/step - loss: 0.0379 - accuracy: 0.9644 - val_loss: 0.1655 - val_accuracy: 0.7768
Epoch 3/10
83/83 [==============================] - 14s 172ms/step - loss: 0.0348 - accuracy: 0.9670 - val_loss: 0.0824 - val_accuracy: 0.9226
Epoch 4/10
83/83 [==============================] - 14s 172ms/step - loss: 0.0341 - accuracy: 0.9669 - val_loss: 0.0737 - val_accuracy: 0.9284
Epoch 5/10
83/83 [==============================] - 14s 173ms/step - loss: 0.0360 - accuracy: 0.9646 - val_loss: 0.0848 - val_accuracy: 0.9288
Epoch 6/10
83/83 [==============================] - 14s 172ms/step - loss: 0.0342 - accuracy: 0.9664 - val_loss: 0.0580 - val_accuracy: 0.9542
Epoch 7/10
83/83 [==============================] - 14s 172ms/step - loss: 0.0348 - accuracy: 0.9657 - val_loss: 0.0964 - val_accuracy: 0.9173

In [ ]:
y_pred_tr = model.predict([test_seqs_1, test_seqs_2])

In [ ]:
tr_acc = compute_accuracy(Y_test, y_pred_tr)

(2640, 1) (2640, 1, 128)


In [ ]:
tr_acc

0.9073119646392906

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [ ]:
# pred = y_pred_tr.ravel() < 0.5
# pred
y_pred_tr = model.predict([test_seqs_1, test_seqs_2], verbose=1)
y_pred_tr.shape, test_seqs_1.shape

83/83 [==============================] - 2s 20ms/step


((2640, 1, 128), (2640, 1000))

In [ ]:
tr_acc = compute_accuracy(Y_test, y_pred_tr)

(2640, 1) (2640, 1, 128) (337920,)


In [ ]:
preds = (y_pred_tr.ravel() < 0.5)

In [ ]:
results = np.array([[int(x), y[0]] for x, y in zip(preds, Y_test)])

In [ ]:
precision,recall,fscore,support = precision_recall_fscore_support(results[:, 0], results[:, 1], labels=[1])

In [ ]:
precision,recall,fscore,support 

(array([0.08163265]), array([0.04347826]), array([0.05673759]), array([184]))